In [1]:
import os

In [2]:
from glob import glob

In [3]:
import matplotlib.pyplot as plt

In [4]:
import random

In [5]:
import tensorflow as tf

In [6]:
from tensorflow import keras

In [7]:
import pandas as pd

In [8]:
from PIL import Image

In [9]:
import tensorflow.data as tfd

In [10]:
import plotly.express as px

In [ ]:
!pip install -q git+https://github.com/tensorflow/examples.git

In [ ]:
from tensorflow_examples.models.pix2pix import pix2pix

In [ ]:
from tensorflow.keras import layers

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
import tensorflow

In [ ]:
import pickle

In [ ]:
IMAGE_DIR = "Dataset/Forest Segmented/images/"
MASK_DIR = "Dataset/Forest Segmented/masks/"

In [ ]:
SEED = 42
BATCH_SIZE = 32

In [ ]:
BATCH_SIZE = 16
IMAGE_SIZE = (128, 128)

In [ ]:
VALIDATION_SIZE = int(1e3)
AUTOTUNE = tfd.AUTOTUNE
tf.random.set_seed(SEED)

In [ ]:
def decode_image(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.clip_by_value(image, clip_value_min = 0.0, clip_value_max = 1.0)
    return image

In [ ]:
def decode_masks(file_path):
    mask_image = decode_image(file_path)
    mask_image = mask_image[:, :, :1]  # Keep only the first channel for binary masks
    return mask_image

In [ ]:
def show_image(image, title=None, alpha=1):
    plt.imshow(image, alpha=alpha)
    plt.title(title)
    plt.axis('off')

In [ ]:
def show_image_masks(dataset, n_rows=5, n_cols=3, figsize=(10, 15)):
    images, masks = next(iter(dataset.take(1)))

    plt.figure(figsize=figsize)
    for i in range(1, (n_rows * n_cols) + 1, n_cols):

        plt.subplot(n_rows, n_cols, i)
        show_image(images[i], title="Aerial Imagery")

        plt.subplot(n_rows, n_cols, i + 1)
        show_image(masks[i], title="Image Mask")

        plt.subplot(n_rows, n_cols, i + 2)
        show_image(images[i])
        show_image(masks[i], alpha=0.5, title="Overlapping")

    plt.show()

In [ ]:
metadata = pd.read_csv("Dataset/meta_data.csv")

In [ ]:
metadata.image = metadata.image.apply(lambda file_name: f"{IMAGE_DIR}{file_name}")

In [ ]:
metadata['mask'] = metadata['mask'].apply(lambda file_name: f"{MASK_DIR}{file_name}")

In [ ]:
metadata

In [ ]:
image_files = tfd.Dataset.from_tensor_slices(metadata['image'])
mask_files = tfd.Dataset.from_tensor_slices(metadata['mask'])

In [ ]:
images = image_files.map(decode_image)
masks = mask_files.map(decode_masks)

In [ ]:
dataset = tf.data.Dataset.zip((images, masks))
dataset = dataset.repeat(2).shuffle(1000).batch(BATCH_SIZE)

In [ ]:
show_image_masks(dataset)

In [ ]:
class Augment(layers.Layer):
    def __init__(self, seed=42):
        super().__init__()
        
        self.augment_images = tf.keras.Sequential([
            layers.RandomRotation(0.3, seed=seed),
            layers.RandomFlip(mode="horizontal", seed=seed),
        ])
        
        self.augment_masks = tf.keras.Sequential([
            layers.RandomRotation(0.3, seed=seed),
            layers.RandomFlip(mode="horizontal", seed=seed),
        ])
            
    def call(self, inputs, labels):
        inputs = self.augment_images(inputs)
        labels = self.augment_masks(labels)
        return inputs, labels


In [ ]:
augmented_ds = dataset.map(Augment(SEED)).cache().prefetch(AUTOTUNE)

In [ ]:
show_image_masks(augmented_ds)

In [ ]:
total_samples = len(augmented_ds)

In [ ]:
print("Total number of samples in the augmented dataset:", total_samples * BATCH_SIZE)

In [ ]:
valid_ds = augmented_ds.take(VALIDATION_SIZE//BATCH_SIZE).prefetch(AUTOTUNE)
train_ds = augmented_ds.skip(VALIDATION_SIZE//BATCH_SIZE).prefetch(AUTOTUNE)

In [ ]:
train_samples = tfd.experimental.cardinality(train_ds).numpy() * BATCH_SIZE
valid_samples = tfd.experimental.cardinality(valid_ds).numpy() * BATCH_SIZE

In [ ]:
print("Training   Samples:", train_samples)
print("Validation Samples:", valid_samples)

In [ ]:
base_model = tensorflow.keras.applications.MobileNetV2(
    input_shape=(*IMAGE_SIZE, 3),
    include_top=False
)

In [ ]:
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]

In [ ]:
base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

In [ ]:
encoder = tensorflow.keras.Model(
    inputs=base_model.input,
    outputs=base_model_outputs,
    name="Encoder"
)

In [ ]:
encoder.trainable = False

In [ ]:
decoder = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

In [ ]:
def create_model(encoder_block, decoder_block, output_channels=1, **kwargs):
    inputs = keras.Input(shape=(*IMAGE_SIZE, 3), name="ImageInput")
    encodings = encoder_block(inputs)
    skips = reversed(encodings[:-1])        
    encoding = encodings[-1]
    for index, (up, skip) in enumerate(zip(decoder_block, skips)):
        encoding = up(encoding)
        encoding = layers.Concatenate(name=f"ConCat-{index+1}")([encoding, skip])
    mask_out = layers.Conv2DTranspose(
        filters=1,
        kernel_size=3,
        strides=2,
        padding="same",
    )(encoding)
    
    mask_out = layers.Conv2D(
        filters=1,
        kernel_size=1,
        padding="same",
        activation='sigmoid',
        name="MaskOut"
    )(mask_out)
    model = keras.Model(inputs, mask_out, **kwargs)
    
    return model

In [ ]:
model = create_model(encoder, decoder, name="create_model")
model.summary()

In [ ]:
def show_pred_masks(dataset, model, n_rows=5, n_cols=6, figsize=(15, 15), savepath=None):
    images, masks = next(iter(dataset.take(1)))
    pred_masks = model.predict(images)

    plt.figure(figsize=figsize)
    for i in range(1, (n_rows * n_cols) + 1, n_cols):
        
        plt.subplot(n_rows, n_cols, i)
        show_image(images[i], title="Aerial Imagery")

        plt.subplot(n_rows, n_cols, i + 1)
        show_image(masks[i], title="True Image Mask")

        plt.subplot(n_rows, n_cols, i + 2)
        show_image(pred_masks[i], title="Predicted Image Mask")
        
        plt.subplot(n_rows, n_cols, i + 3)
        show_image(tf.cast(pred_masks[i]>=0.5, tf.float32), title="Processed Mask(0.5)")
        
        plt.subplot(n_rows, n_cols, i + 4)
        show_image(tf.cast(pred_masks[i]>=0.7, tf.float32), title="Processed Mask(0.7)")
        
        plt.subplot(n_rows, n_cols, i + 5)
        show_image(images[i])
        show_image(pred_masks[i], alpha=0.5, title="Overlapping")
        
        
    if savepath is not None:
        plt.savefig(savepath)
        
    plt.show()

In [ ]:
class ShowImageMasksCallback(keras.callbacks.Callback):
    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset

    def on_epoch_end(self, epoch, logs=None):
        show_pred_masks(dataset=self.dataset, n_rows=3, model=self.model, savepath=f"ModelPred_Epoch_{epoch}.png", figsize=(15, 10))

In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=['accuracy']
)

In [ ]:
steps_per_epoch = train_samples//BATCH_SIZE

In [ ]:
can_train = False
epochs = 1

In [ ]:
if can_train:
    history = model.fit(
    train_ds,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_ds,
    epochs=epochs,
    callbacks=[ShowImageMasksCallback(valid_ds)]
)
    model.save("Aerial-Forest-Segmentation.keras")
    with open("Aerial-Forest-Segmentation.pickle", "wb") as fs:
        pickle.dump(history.history)
    history = history.history
else:
    model = load_model("Aerial-Forest-Segmentation.keras")
    with open("Aerial-Forest-Segmentation.pickle", "rb") as fs:
        history = pickle.load(fs)

In [ ]:
show_pred_masks(valid_ds.shuffle(500), model=model, n_rows=3)

In [ ]:
show_pred_masks(valid_ds.shuffle(500), model=model, n_rows=3)

In [ ]:
show_pred_masks(valid_ds.shuffle(500), model=model, n_rows=3)

In [ ]:
figsize = (20, 20)
n_rows, n_cols = 3, 5

images, masks = next(iter(train_ds.shuffle(1000).take(1)))
pred_masks = model.predict(images)

plt.figure(figsize=figsize)
for i in range(1, (n_rows * n_cols) + 1, n_cols):

    plt.subplot(n_rows, n_cols, i)
    show_image(images[i], title="Aerial Imagery")
    
    plt.subplot(n_rows, n_cols, i + 1)
    show_image(images[i])
    show_image(masks[i], alpha=0.6, title="True Mask")
    
    plt.subplot(n_rows, n_cols, i + 2)
    show_image(images[i])
    show_image(pred_masks[i]>=0.5, alpha=0.6, title="Overlapping (0.5)")
    
    plt.subplot(n_rows, n_cols, i + 3)
    show_image(images[i])
    show_image(pred_masks[i]>=0.7, alpha=0.6, title="Overlapping (0.7)")
    
    plt.subplot(n_rows, n_cols, i + 4)
    show_image(images[i])
    show_image(pred_masks[i]>=0.9, alpha=0.6, title="Overlapping (0.9)")

plt.show()